In [1]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, LimitOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderType
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.trading.requests import GetOptionContractsRequest

# Define your API keys
api_key = "PKE6XBHTF3BZDWYWM7B6"
secret_key = "rDmFZvisA5TcHqsKO3dcBVNmNK5rjHDRwsGXFvSz"

# Set up the Alpaca trading client for paper trading
paper = True
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper)

# Example: Fetch Option Contracts
option_data_client = OptionHistoricalDataClient(api_key=api_key, secret_key=secret_key)

c:\Users\Public\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Public\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:


# # Example Request for Option Contracts
# contracts_request = GetOptionContractsRequest(
#     symbol="AAPL", # Symbol for the asset (e.g., Apple)
#     expiration_date="2024-10-20", # Set expiration date
#     contract_type="call", # Choose either 'call' or 'put'
# )

# option_contracts = trade_client.get_option_contracts(contracts_request)

# # Print Option Contracts
# for contract in option_contracts:
#     print(contract)

# Example: Place a market order for an option contract
order_request = MarketOrderRequest(
    symbol="X",  # Underlying symbol
    qty=1,          # Number of option contracts
    side=OrderSide.BUY,
    time_in_force=TimeInForce.GTC  # Good Till Canceled
)

order = trade_client.submit_order(order_request)
print(order)



id=UUID('380c1947-11e3-4060-bf8a-fbb96466b567') client_order_id='1d3031bb-afe4-4254-941c-b47fface04be' created_at=datetime.datetime(2024, 12, 12, 12, 57, 46, 814467, tzinfo=TzInfo(UTC)) updated_at=datetime.datetime(2024, 12, 12, 12, 57, 46, 815511, tzinfo=TzInfo(UTC)) submitted_at=datetime.datetime(2024, 12, 12, 12, 57, 46, 814467, tzinfo=TzInfo(UTC)) filled_at=None expired_at=None canceled_at=None failed_at=None replaced_at=None replaced_by=None replaces=None asset_id=UUID('4ea43090-b5be-42bd-a4a8-f4607616fe0e') symbol='X' asset_class=<AssetClass.US_EQUITY: 'us_equity'> notional=None qty='1' filled_qty='0' filled_avg_price=None order_class=<OrderClass.SIMPLE: 'simple'> order_type=<OrderType.MARKET: 'market'> type=<OrderType.MARKET: 'market'> side=<OrderSide.BUY: 'buy'> time_in_force=<TimeInForce.GTC: 'gtc'> limit_price=None stop_price=None status=<OrderStatus.PENDING_NEW: 'pending_new'> extended_hours=False legs=None trail_percent=None trail_price=None hwm=None


In [5]:
# Example: Get order info using order_id
order_id = "318c168f-88de-4987-b6e8-1b8466a81e93"
#trade_client.cancel_order_by_id(order_id)
# Fetch order details by ID
order = trade_client.get_order_by_id(order_id)

# Print the order details
print(f"Order ID: {order.id}")
print(f"Status: {order.status}")
print(f"Symbol: {order.symbol}")
print(f"Quantity: {order.qty}")
print(f"Filled Quantity: {order.filled_qty}")
print(f"Type: {order.order_type}")
print(f"Side: {order.side}")
print(f"Submitted At: {order.submitted_at}")
print(f"Filled At: {order.filled_at if order.filled_at else 'Not yet filled'}")

Order ID: 318c168f-88de-4987-b6e8-1b8466a81e93
Status: filled
Symbol: X
Quantity: 1
Filled Quantity: 1
Type: market
Side: sell
Submitted At: 2024-11-08 17:49:24.996279+00:00
Filled At: 2024-11-08 17:49:25.418652+00:00


In [6]:
print(trade_client.get_all_positions())

[]


Selling Order Working

In [4]:
# Define the symbol and the quantity you want to close
symbol = "X"  # Replace with your position symbol
quantity_to_close = 1  # Quantity to close (must be <= current position size)

# Fetch the current position to ensure the quantity is valid
position = trade_client.get_open_position(symbol)
print(f"Current Position Size: {position.qty}")

if float(position.qty) >= quantity_to_close:
    # Create a market order to close the specified quantity
    close_order = MarketOrderRequest(
        symbol=symbol,
        qty=quantity_to_close,
        side=OrderSide.SELL if float(position.qty) > 0 else OrderSide.BUY,  # Sell if long, buy if short
        time_in_force=TimeInForce.DAY,
    )

    # Submit the order
    order = trade_client.submit_order(close_order)
    print(f"Order to close {quantity_to_close} shares of {symbol} submitted.")
    print(f"Order ID: {order.id}, Status: {order.status}")

else:
    print(f"Not enough quantity to close. Current size is {position.qty}.")

Current Position Size: 1
Order to close 1 shares of X submitted.
Order ID: 318c168f-88de-4987-b6e8-1b8466a81e93, Status: pending_new


In [9]:
# Fetch all open positions
positions = trade_client.get_all_positions()
if not positions:
    print("No open positions found.")
else:
    for pos in positions:
        print(f"Position: {pos.symbol}, Qty: {pos.qty}, Side: {pos.side}, "
              f"Market Value: {pos.market_value}, Avg Entry Price: {pos.avg_entry_price}")

# Fetch all open orders (only those with 'open' status)
orders = trade_client.get_orders()
open_orders = [order for order in orders if order.status == 'open']

if not open_orders:
    print("No open orders found.")
else:
    for order in open_orders:
        print(f"Order ID: {order.id}, Symbol: {order.symbol}, Side: {order.side}, "
              f"Qty: {order.qty}, Status: {order.status}, Type: {order.order_type}")

Position: META, Qty: 3.716068348, Side: long, Market Value: 2335.548957, Avg Entry Price: 580.393333
Position: NLY, Qty: 124.285714285, Side: long, Market Value: 2455.885714, Avg Entry Price: 19.673333
Position: OTIS, Qty: 27.523857358, Side: long, Market Value: 2686.603717, Avg Entry Price: 100.473333
Position: PATK, Qty: 23.412818955, Side: long, Market Value: 3166.817892, Avg Entry Price: 131.18
Position: PPG, Qty: 28.382027989, Side: long, Market Value: 3479.352811, Avg Entry Price: 123.18
Position: SGRY, Qty: 129.591497841, Side: long, Market Value: 2721.421455, Avg Entry Price: 26.55
Position: SWKS, Qty: 51.769285208, Side: long, Market Value: 4509.104742, Avg Entry Price: 83.58
Position: TER, Qty: 46.115054165, Side: long, Market Value: 5533.8065, Avg Entry Price: 106.95
Position: XRX, Qty: 637.772675086, Side: long, Market Value: 5644.288175, Avg Entry Price: 8.503333
No open orders found.


In [3]:
# Assuming you have the trade_client object initialized
all_orders = trade_client.get_orders()

for order in all_orders:
    if order.status == "new":
        trade_client.cancel_order_by_id(order.id)
    print(f"Canceled order {order.id}, status: {order.status}, filled: {order.filled_qty}")

In [5]:
trade_client.close_all_positions()

[]